In [4]:
import pandas as pd
import networkx as nx
import ast
import numpy as np
import seaborn as sns
from matplotlib.pylab import plt
import itertools
import community
%matplotlib inline

In [87]:
#a = pd.read_csv("./the-movies-dataset/credits.csv")
credits = pd.read_csv("./the-movies-dataset/credits.csv")
actors = pd.read_csv("./actors.csv")
metadata = pd.read_csv("./the-movies-dataset/movies_metadata.csv")
metadata_set_index = metadata.set_index('id')
actors_set_index = actors.set_index('credit_id')

/Users/user1/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
actors_sorted = actors.sort_values(by = ['appearance'], ascending = False)
#actors_sorted.head()

In [89]:
credits.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [101]:
metadata.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


映画ごとの、キャストのリストを取得する

In [6]:
## データフレーム中のstringをキャスト(dict型)のリストに変換する
def make_list_from_str(string):
    if isinstance(string, str):
        result_list = eval(string)
    else:
        result_list = np.nan
    return result_list

In [45]:
credits.cast = credits.cast.map(make_list_from_str)
credits.crew = credits.crew.map(make_list_from_str)

In [119]:
#metadata = metadata.drop([19730,29503,35587])
metadata.id = pd.to_numeric(metadata.id)
metadata.release_date = pd.to_datetime(metadata['release_date'], format='%Y-%m-%d', errors='ignore')

映画+キャストのデータセットを新しいものに制限する

In [132]:
def cre_mov_pickup(datetime):
    c_m = pd.merge(credits, metadata, right_on ='id' ,left_on='id', how = 'left')
    c_m_sorted = c_m.sort_values(by = ['release_date'], ascending = False)
    c_m_sorted_pickup = c_m_sorted[c_m_sorted['release_date'] > pd.datetime.strptime(datetime,'%Y-%m-%d')]
    movies_cast_list = c_m_sorted_pickup["cast"].tolist()
    return movies_cast_list

ピックアップしたいアクターのリストを取得する

In [10]:
def make_actors_list(min_appearance):
    actors_sorted = actors.sort_values(by = ['appearance'], ascending = False)
    actors_pickup = actors_sorted[actors_sorted['appearance'] > 10]
    pickup_actors_list = actors_pickup['credit_id'].tolist()
    return pickup_actors_list

In [70]:
#キャストと映画の配列を作る（処理には使ってない）
castmoviearr = np.zeros((206158,45476))
castmovie = pd.DataFrame(castmoviearr, columns=credits["id"].tolist(), index=actors_info["credit_id"].tolist())
castmovie2 = pd.merge(actors_pickup, castmovie, right_index =True , left_on='credit_id', how = 'inner')
castmovie2.shape

共起リストを作る

In [140]:
def makekyoukilist(movies_cast_list, pickup_actors_list):
    castkyoukilist = []
    for movie in movies_cast_list:
        castlist = []
        movie_dict = eval(movi)
        for cast in movie_dict:
            if cast['credit_id'] in pickup_actors_list:
                castlist.append(cast['credit_id'])
        castkyoukilist.append(castlist)
    return castkyoukilist

アクターの情報を取得する関数

In [77]:
def get_actor_tag(creditid, tagname):
    tagvalue = actors_set_index.loc[creditid][tagname]
    #print(type(tagvalue))
    return tagvalue

ネットワーク化する

In [80]:
def makeGraph(castkyoukilist):
    G = nx.Graph()
    for castlist in castkyoukilist:
        addNodes(G, castlist)
        addEdges(G, castlist)
    return G

def addNodes(G,castlist):
    for cast in castlist:
        #G.add_node(get_actor_tag(cast,'name'), size = get_actor_tag(cast,'appearance'))
        G.add_node(cast, attr_dict = {'name':get_actor_tag(cast,'name'), 'appearance':int(get_actor_tag(cast,'appearance'))})
    return True

def addEdges(G,castlist):
    for node0,node1 in itertools.combinations(castlist, 2):
        if not G.has_node(node0) or not G.has_node(node1):
            continue
        if G.has_edge(node0, node1):
            G.edge[node0][node1]["weight"] += 1
        else:
            G.add_edge(node0, node1, {"weight":1})
    

In [33]:
def clusteringplot(G,partition):                                                                                                           
    size = float(len(set(partition.values())))                                                                                                 
    pos = nx.spring_layout(G)                                                                                                                  
    count = 0.                                                                                                                                 
    for com in set(partition.values()):                                                                                                        
        count += 1.                                                                                                                              
        list_nodes = [nodes for nodes in partition.keys() if partition[nodes] == com] 
        #node_size = [ d["count"]*20 for (n,d) in G.nodes(data=True)]
        nx.draw_networkx_nodes(G, pos, list_nodes, node_color = str(count/size), node_size=30)   
        nx.draw_networkx_labels(G, pos, fontsize=14, font_family="Yu Gothic", font_weight="bold")
    plt.show()

In [19]:
def dividecluster(partition):
    valuelist = list(set(partition.values()))
    num = len(valuelist)
    arr = [[] for i in range(num)]
    for k,v in partition.items():
        arr[v].append(k)
    return arr

全て実行し、グラフを出力

In [141]:
pickup_actors_list = make_actors_list(10)
movies_cast_list = cre_mov_pickup('2000-01-01')
kyoukilist = makekyoukilist(movies_cast_list, pickup_actors_list)

AttributeError: 'str' object has no attribute 'eval'

In [81]:
G = makeGraph(kyoukilist)

In [67]:
print(G.number_of_nodes())
print(G.number_of_edges())

9144
21425


In [73]:
G.node['52fe45d3c3a36847f80dcb1b']

{'appearance': 110, 'name': 'Gérard Depardieu'}

In [82]:
nx.write_gexf(G, "casts_3.gexf")

その他の処理(クラスタリングとか)

In [49]:
partition = community.best_partition(G)
#clusteringplot(G,partition)

#ノードに名前つけるのと、共演回数でエッジの重みを変えるのがまだ

In [50]:
arr = dividecluster(partition)
with open('clustering.txt','w') as f:
    for cluster in arr:
        print(arr.index(cluster))
        print(cluster)

0
['52fe4284c3a36847f8024f95', '52fe4284c3a36847f8024f99', '52fe4284c3a36847f8024f9d', '52fe4284c3a36847f8024fa5', '52fe4284c3a36847f8024fa9', '52fe4284c3a36847f8024fb5', '52fe4284c3a36847f8024fb9']
1
['52fe44bfc3a36847f80a7c73', '52fe44bfc3a36847f80a7c77', '52fe44bfc3a36847f80a7c9d', '52fe44c0c3a36847f80a7ceb', '52fe44c0c3a36847f80a7cef', '52fe44bfc3a36847f80a7ca1']
2
['52fe466a9251416c75077a8d', '52fe466a9251416c75077a91', '52fe466a9251416c75077a95', '52fe466a9251416c75077a99', '52fe466a9251416c75077a9d', '53e5fcc2c3a3684430000d65', '53e5fcd4c3a3684433000e1a']
3
['52fe44779251416c91011ab1', '52fe44779251416c91011ab5', '52fe44779251416c91011ac1', '52fe44779251416c91011ac5', '52fe44779251416c91011acf', '56f8a929c3a36816e80084f3']
4
['52fe44959251416c75039eb9', '52fe44959251416c75039ebd', '52fe44959251416c75039ec1', '52fe44959251416c75039ef3', '52fe44959251416c75039ef7', '52fe44959251416c75039efb', '52fe44959251416c75039f07', '52fe44959251416c75039f0b']
5
['52fe4292c3a36847f80291f5', '5

422
['52fe454e9251416c75052035']
423
['52fe43729251416c75010a47']
424
['52fe43f29251416c75023d3b']
425
['52fe45119251416c9102661d']
426
['55104902c3a368487400dca3']
427
['52fe44ac9251416c7503cfb9', '56888fa2c3a3684be9023f67', '56888fcac3a36860e90252e9', '56889001c3a3684be9023f76', '56889086c3a3684be9023f90', '5688910fc3a3684be9023f9e', '547a51f49251414c65000478']
428
['5791d9a79251415fd8005657']
429
['52fe463d9251416c75071d99', '52fe463d9251416c75071da9', '52fe463d9251416c75071db1']
430
['52fe44419251416c7502e405']
431
['52fe4810c3a36847f8156755', '5414221e0e0a26198a0018f7']
432
['52fe44879251416c7503815b', '52fe44889251416c7503815f', '52fe44889251416c75038183']
433
['52fe44819251416c9101324d', '52fe44819251416c91013255']
434
['52fe45d7c3a36847f80dd903']
435
['52fe4857c3a36847f8162977']
436
['52fe4472c3a368484e02377b']
437
['52fe47369251416c75091835']
438
['52fe43cb9251416c7501e823']
439
['54f94dc9c3a36878e1000f55']
440
['59007eb792514161a600b044', '59007e5992514161be00a3ae', '59007e47

['52fe442ac3a368484e01358b']
1195
['52fe444b9251416c7502fc07']
1196
['54624db8c3a36808200000bd']
1197
['5452be01c3a368092700029a']
1198
['52fe4b489251416c750fdf3b']
1199
['52fe46479251416c9104f4b9', '57c5fb2f925141214d000338']
1200
['52fe456f9251416c7505673d', '52fe456f9251416c75056741', '557fbb70925141256b000a1f']
1201
['52fe44e2c3a36847f80afcd3', '52fe44e2c3a36847f80afcdb', '52fe44e2c3a36847f80afc91', '57538e0dc3a3680e820015c7', '53ba7bfbc3a368072e00405d']
1202
['52fe4982c3a368484e12f343']
1203
['52fe4530c3a368484e04d25d']
1204
['52fe44e2c3a368484e03c86d', '52fe44e2c3a368484e03c871']
1205
['52fe44869251416c75037cd5']
1206
['52fe425ac3a36847f8017873', '52fe425ac3a36847f8017897', '55e4f58d92514137a2000552', '55e502d1c3a36813ed00090e']
1207
['52fe482dc3a36847f815b503', '52fe482dc3a36847f815b50f']
1208
['52fe43e2c3a368484e003b0d', '56739bed9251412dab0011c6']
1209
['52fe4310c3a36847f80378d7']
1210
['52fe451bc3a36847f80bcd0b', '52fe451bc3a36847f80bcd13']
1211
['52fe4275c3a36847f8020269']
1

Cytoscapeか何かで可視化する用

networkxのHPに大体のできることが書いてある
https://networkx.github.io/documentation/networkx-1.9/reference/functions.html

In [18]:
labels = dict([(i, str(i)) for i in range(nx.number_of_nodes(G))])
nx.set_node_attributes(G, 'community', partition)
#nx.set_node_attributes(G, 'label', labels)
nx.write_gml(G, "community.gml")

NameError: name 'partition' is not defined